In [0]:
%pip install nltk

Python interpreter will be restarted.
Python interpreter will be restarted.


In [0]:
import nltk # Library for natural language processing
import string
import math
from nltk.corpus import stopwords # To import a list of common stopwords (e.g., 'and', 'the')
from nltk.tokenize import word_tokenize # tokenize text into individual words
#downloading stopwords data set
nltk.download('stopwords')
#downloading tokenizer model
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Out[1]: True

In [0]:
# Importing source file
mov_summaries = sc.textFile("dbfs:/FileStore/plot_summaries.txt")
mov_summaries.collect()

Out[2]: ["23890098\tShlykov, a hard-working taxi driver and Lyosha, a saxophonist, develop a bizarre love-hate relationship, and despite their prejudices, realize they aren't so different after all.",
 '31186339\tThe nation of Panem consists of a wealthy Capitol and twelve poorer districts. As punishment for a past rebellion, each district must provide a boy and girl  between the ages of 12 and 18 selected by lottery  for the annual Hunger Games. The tributes must fight to the death in an arena; the sole survivor is rewarded with fame and wealth. In her first Reaping, 12-year-old Primrose Everdeen is chosen from District 12. Her older sister Katniss volunteers to take her place. Peeta Mellark, a baker\'s son who once gave Katniss bread when she was starving, is the other District 12 tribute. Katniss and Peeta are taken to the Capitol, accompanied by their frequently drunk mentor, past victor Haymitch Abernathy. He warns them about the "Career" tributes who train intensively at special 

In [0]:
# Split a document into doc ID and the text
def doc_Splitting(doc):
    splitDoc = doc.split("\t")
    docID= [splitDoc[0],splitDoc[1]] # Store the first part as doc ID and second as text
    return docID
# Then mapping to a tuple where the first element is the document ID and the second is the text
doc_Data = mov_summaries.map(lambda x: doc_Splitting(x)).map(lambda x: (x[0],x[1]))
doc_Data.take(5)

Out[3]: [('23890098',
  "Shlykov, a hard-working taxi driver and Lyosha, a saxophonist, develop a bizarre love-hate relationship, and despite their prejudices, realize they aren't so different after all."),
 ('31186339',
  'The nation of Panem consists of a wealthy Capitol and twelve poorer districts. As punishment for a past rebellion, each district must provide a boy and girl  between the ages of 12 and 18 selected by lottery  for the annual Hunger Games. The tributes must fight to the death in an arena; the sole survivor is rewarded with fame and wealth. In her first Reaping, 12-year-old Primrose Everdeen is chosen from District 12. Her older sister Katniss volunteers to take her place. Peeta Mellark, a baker\'s son who once gave Katniss bread when she was starving, is the other District 12 tribute. Katniss and Peeta are taken to the Capitol, accompanied by their frequently drunk mentor, past victor Haymitch Abernathy. He warns them about the "Career" tributes who train intensively 

In [0]:
# Remove stop words for each doc
#imposrting all stop words
stopWords = set(stopwords.words('english'))

def removeStopWords(docs):
    
    wordDocs = docs.split(' ')
    filter_words = []
    for word in wordDocs:
        word1 = word.lower() #converting all the words into lower case
        word2 = ""
        for l in word1:
            # Check if (l) in a word is alpha numeric or not
            if l.isalnum():
                word2 += l #if its alphanumeric append it to a string
        if word2 != '':
            filter_words.append(word2)

    finalWords = []

    # Remove stop words from the final list of tokenized words
    for w in filter_words:
        if w not in stopWords:
            finalWords.append(w) #keep words that are not in stop words list
    return finalWords
# flatMapValues applies the function to the value (text) part of the (docID, text) tuple
# map to (docID, cleaned words) pairs, followed by adding a count (1) to each pair
cleaned_Data = doc_Data.flatMapValues(lambda x : removeStopWords(x)).map(lambda x: (x,1))

In [0]:
# cleaned_Data - ((ID,term), count = 1)
cleaned_Data.collect()

Out[5]: [(('23890098', 'shlykov'), 1),
 (('23890098', 'hardworking'), 1),
 (('23890098', 'taxi'), 1),
 (('23890098', 'driver'), 1),
 (('23890098', 'lyosha'), 1),
 (('23890098', 'saxophonist'), 1),
 (('23890098', 'develop'), 1),
 (('23890098', 'bizarre'), 1),
 (('23890098', 'lovehate'), 1),
 (('23890098', 'relationship'), 1),
 (('23890098', 'despite'), 1),
 (('23890098', 'prejudices'), 1),
 (('23890098', 'realize'), 1),
 (('23890098', 'arent'), 1),
 (('23890098', 'different'), 1),
 (('31186339', 'nation'), 1),
 (('31186339', 'panem'), 1),
 (('31186339', 'consists'), 1),
 (('31186339', 'wealthy'), 1),
 (('31186339', 'capitol'), 1),
 (('31186339', 'twelve'), 1),
 (('31186339', 'poorer'), 1),
 (('31186339', 'districts'), 1),
 (('31186339', 'punishment'), 1),
 (('31186339', 'past'), 1),
 (('31186339', 'rebellion'), 1),
 (('31186339', 'district'), 1),
 (('31186339', 'must'), 1),
 (('31186339', 'provide'), 1),
 (('31186339', 'boy'), 1),
 (('31186339', 'girl'), 1),
 (('31186339', 'ages'), 1),


In [0]:
# terms_Count = ((ID,term), count)
terms_Count = cleaned_Data.reduceByKey(lambda x,y: x+y)
terms_Count.collect()

Out[6]: [(('23890098', 'hardworking'), 1),
 (('23890098', 'lyosha'), 1),
 (('23890098', 'saxophonist'), 1),
 (('23890098', 'develop'), 1),
 (('23890098', 'lovehate'), 1),
 (('23890098', 'relationship'), 1),
 (('23890098', 'realize'), 1),
 (('23890098', 'arent'), 1),
 (('31186339', 'nation'), 1),
 (('31186339', 'panem'), 1),
 (('31186339', 'twelve'), 1),
 (('31186339', 'poorer'), 1),
 (('31186339', 'past'), 2),
 (('31186339', 'rebellion'), 1),
 (('31186339', 'must'), 2),
 (('31186339', 'provide'), 2),
 (('31186339', 'boy'), 2),
 (('31186339', '12'), 4),
 (('31186339', 'selected'), 1),
 (('31186339', 'games'), 4),
 (('31186339', 'tributes'), 4),
 (('31186339', 'fight'), 1),
 (('31186339', 'arena'), 2),
 (('31186339', 'sole'), 1),
 (('31186339', 'survivor'), 2),
 (('31186339', 'rewarded'), 1),
 (('31186339', 'wealth'), 1),
 (('31186339', '12yearold'), 1),
 (('31186339', 'primrose'), 1),
 (('31186339', 'sister'), 1),
 (('31186339', 'volunteers'), 1),
 (('31186339', 'take'), 1),
 (('3118633

In [0]:
# Calculating the Term frequency(tf) for each doc and each term
tf = terms_Count.map(lambda x: (x[0][1],(x[0][0],x[1])))
tf.collect()

Out[7]: [('hardworking', ('23890098', 1)),
 ('lyosha', ('23890098', 1)),
 ('saxophonist', ('23890098', 1)),
 ('develop', ('23890098', 1)),
 ('lovehate', ('23890098', 1)),
 ('relationship', ('23890098', 1)),
 ('realize', ('23890098', 1)),
 ('arent', ('23890098', 1)),
 ('nation', ('31186339', 1)),
 ('panem', ('31186339', 1)),
 ('twelve', ('31186339', 1)),
 ('poorer', ('31186339', 1)),
 ('past', ('31186339', 2)),
 ('rebellion', ('31186339', 1)),
 ('must', ('31186339', 2)),
 ('provide', ('31186339', 2)),
 ('boy', ('31186339', 2)),
 ('12', ('31186339', 4)),
 ('selected', ('31186339', 1)),
 ('games', ('31186339', 4)),
 ('tributes', ('31186339', 4)),
 ('fight', ('31186339', 1)),
 ('arena', ('31186339', 2)),
 ('sole', ('31186339', 1)),
 ('survivor', ('31186339', 2)),
 ('rewarded', ('31186339', 1)),
 ('wealth', ('31186339', 1)),
 ('12yearold', ('31186339', 1)),
 ('primrose', ('31186339', 1)),
 ('sister', ('31186339', 1)),
 ('volunteers', ('31186339', 1)),
 ('take', ('31186339', 1)),
 ('peeta', 

In [0]:
# Calculate document frequency(df)
initial_Df = terms_Count.map(lambda x : (x[0][1],(x[0][0],x[1],1)))
initial_Df.collect()

Out[8]: [('hardworking', ('23890098', 1, 1)),
 ('lyosha', ('23890098', 1, 1)),
 ('saxophonist', ('23890098', 1, 1)),
 ('develop', ('23890098', 1, 1)),
 ('lovehate', ('23890098', 1, 1)),
 ('relationship', ('23890098', 1, 1)),
 ('realize', ('23890098', 1, 1)),
 ('arent', ('23890098', 1, 1)),
 ('nation', ('31186339', 1, 1)),
 ('panem', ('31186339', 1, 1)),
 ('twelve', ('31186339', 1, 1)),
 ('poorer', ('31186339', 1, 1)),
 ('past', ('31186339', 2, 1)),
 ('rebellion', ('31186339', 1, 1)),
 ('must', ('31186339', 2, 1)),
 ('provide', ('31186339', 2, 1)),
 ('boy', ('31186339', 2, 1)),
 ('12', ('31186339', 4, 1)),
 ('selected', ('31186339', 1, 1)),
 ('games', ('31186339', 4, 1)),
 ('tributes', ('31186339', 4, 1)),
 ('fight', ('31186339', 1, 1)),
 ('arena', ('31186339', 2, 1)),
 ('sole', ('31186339', 1, 1)),
 ('survivor', ('31186339', 2, 1)),
 ('rewarded', ('31186339', 1, 1)),
 ('wealth', ('31186339', 1, 1)),
 ('12yearold', ('31186339', 1, 1)),
 ('primrose', ('31186339', 1, 1)),
 ('sister', ('31

In [0]:
#Calculate each term count from the document (term, count)
total_Docs_Count = mov_summaries.count()
each_Term_Count = initial_Df.map(lambda x : (x[0],x[1][2])).reduceByKey(lambda x,y : x+y)
each_Term_Count.collect()

Out[9]: [('insects', 79),
 ('name', 3382),
 ('assumed', 267),
 ('ceiling', 265),
 ('reads', 765),
 ('mickey', 229),
 ('communitys', 29),
 ('lucys', 70),
 ('colby', 22),
 ('shock', 801),
 ('brutality', 96),
 ('regiment', 125),
 ('us', 1938),
 ('realized', 323),
 ('letting', 501),
 ('henrion', 1),
 ('mose', 4),
 ('antics', 224),
 ('ultrasonicus', 1),
 ('pole', 297),
 ('nosecone', 2),
 ('fourth', 406),
 ('swim', 322),
 ('lost', 3044),
 ('east', 640),
 ('shiva', 86),
 ('france', 753),
 ('aunts', 127),
 ('isolated', 454),
 ('windy', 20),
 ('spirit', 899),
 ('moneyold', 1),
 ('decompression', 16),
 ('beach', 1033),
 ('filmed', 315),
 ('stuffs', 60),
 ('statement', 197),
 ('smiling', 198),
 ('gostopgo', 1),
 ('stations', 155),
 ('enter', 1587),
 ('nowunited', 2),
 ('tribes', 113),
 ('hang', 366),
 ('louises', 30),
 ('beneath', 424),
 ('askatasuna', 1),
 ('basis', 200),
 ('alcohol', 413),
 ('healthy', 165),
 ('14', 204),
 ('japan', 478),
 ('namehttpwwwmtvcomnewsarticles159673620081009spearsbri

In [0]:
# Calculate inverse document frequency (idf) log(total number of documents/no of documents containing the term)
import math
idf = each_Term_Count.map(lambda x: (x[0], math.log10(total_Docs_Count / x[1])))
idf.collect()

Out[10]: [('insects', 2.7287748737702446),
 ('name', 1.097228361798963),
 ('assumed', 2.199890703696111),
 ('ceiling', 2.203156091123878),
 ('reads', 1.7427405299070684),
 ('mickey', 2.266566482720798),
 ('communitys', 3.16400396716173),
 ('lucys', 2.781303925046429),
 ('colby', 3.28397928423848),
 ('shock', 1.7227694489764485),
 ('brutality', 2.6441307320211176),
 ('regiment', 2.5294919520526298),
 ('us', 1.3390481923459396),
 ('realized', 2.117199442729583),
 ('letting', 1.9265642391934403),
 ('henrion', 4.626401965060686),
 ('mose', 4.0243419737327235),
 ('antics', 2.2761539467265233),
 ('ultrasonicus', 4.626401965060686),
 ('pole', 2.1536455157434737),
 ('nosecone', 4.325371969396705),
 ('fourth', 2.0178759314834918),
 ('swim', 2.118546093364855),
 ('lost', 1.142957316962151),
 ('east', 1.820221991076799),
 ('shiva', 2.6919035138171186),
 ('france', 1.7496069888599854),
 ('aunts', 2.522598244104729),
 ('isolated', 1.9693461122035822),
 ('windy', 3.3253719693967048),
 ('spirit', 1.6

In [0]:
#computing the tf-idf. (term, ((id,tf), idf) 
tf_idf = tf.join(idf)
tf_idf.collect()

Out[11]: [('nation', (('31186339', 1), 2.2684671180602325)),
 ('nation', (('474750', 1), 2.2684671180602325)),
 ('nation', (('15340311', 1), 2.2684671180602325)),
 ('nation', (('32943405', 1), 2.2684671180602325)),
 ('nation', (('20533117', 1), 2.2684671180602325)),
 ('nation', (('17267001', 1), 2.2684671180602325)),
 ('nation', (('6722354', 1), 2.2684671180602325)),
 ('nation', (('33401569', 3), 2.2684671180602325)),
 ('nation', (('22050847', 1), 2.2684671180602325)),
 ('nation', (('26798950', 1), 2.2684671180602325)),
 ('nation', (('4727', 1), 2.2684671180602325)),
 ('nation', (('3546038', 1), 2.2684671180602325)),
 ('nation', (('18436536', 1), 2.2684671180602325)),
 ('nation', (('2395072', 1), 2.2684671180602325)),
 ('nation', (('19392462', 2), 2.2684671180602325)),
 ('nation', (('16963505', 10), 2.2684671180602325)),
 ('nation', (('34613355', 1), 2.2684671180602325)),
 ('nation', (('300972', 9), 2.2684671180602325)),
 ('nation', (('10612031', 1), 2.2684671180602325)),
 ('nation', (

In [0]:
#storing tf-idf into a list (id, (term, tf, idf, tf*idf))
tf_Idf_List = tf_idf.map(lambda x: (x[1][0][0], (x[0], x[1][0][1], x[1][1], x[1][0][1]*x[1][1] )))
tf_Idf_List.collect()

Out[12]: [('31186339', ('nation', 1, 2.2684671180602325, 2.2684671180602325)),
 ('474750', ('nation', 1, 2.2684671180602325, 2.2684671180602325)),
 ('15340311', ('nation', 1, 2.2684671180602325, 2.2684671180602325)),
 ('32943405', ('nation', 1, 2.2684671180602325, 2.2684671180602325)),
 ('20533117', ('nation', 1, 2.2684671180602325, 2.2684671180602325)),
 ('17267001', ('nation', 1, 2.2684671180602325, 2.2684671180602325)),
 ('6722354', ('nation', 1, 2.2684671180602325, 2.2684671180602325)),
 ('33401569', ('nation', 3, 2.2684671180602325, 6.805401354180697)),
 ('22050847', ('nation', 1, 2.2684671180602325, 2.2684671180602325)),
 ('26798950', ('nation', 1, 2.2684671180602325, 2.2684671180602325)),
 ('4727', ('nation', 1, 2.2684671180602325, 2.2684671180602325)),
 ('3546038', ('nation', 1, 2.2684671180602325, 2.2684671180602325)),
 ('18436536', ('nation', 1, 2.2684671180602325, 2.2684671180602325)),
 ('2395072', ('nation', 1, 2.2684671180602325, 2.2684671180602325)),
 ('19392462', ('natio

In [0]:
#calculating and updating the tf_idf(tf_idf=tf * idf)
tf_Idf = tf_idf.map(lambda x: (x[1][0][0],x[0],x[1][0][1]*x[1][1]))
tf_Idf.collect()

Out[13]: [('31186339', 'nation', 2.2684671180602325),
 ('474750', 'nation', 2.2684671180602325),
 ('15340311', 'nation', 2.2684671180602325),
 ('32943405', 'nation', 2.2684671180602325),
 ('20533117', 'nation', 2.2684671180602325),
 ('17267001', 'nation', 2.2684671180602325),
 ('6722354', 'nation', 2.2684671180602325),
 ('33401569', 'nation', 6.805401354180697),
 ('22050847', 'nation', 2.2684671180602325),
 ('26798950', 'nation', 2.2684671180602325),
 ('4727', 'nation', 2.2684671180602325),
 ('3546038', 'nation', 2.2684671180602325),
 ('18436536', 'nation', 2.2684671180602325),
 ('2395072', 'nation', 2.2684671180602325),
 ('19392462', 'nation', 4.536934236120465),
 ('16963505', 'nation', 22.684671180602326),
 ('34613355', 'nation', 2.2684671180602325),
 ('300972', 'nation', 20.416204062542093),
 ('10612031', 'nation', 2.2684671180602325),
 ('14882264', 'nation', 2.2684671180602325),
 ('16972436', 'nation', 2.2684671180602325),
 ('4969625', 'nation', 2.2684671180602325),
 ('30138638', '

In [0]:
def extract_Movie_Data(movie_dataRow):
    #extracting the movie data from each data row
    row = movie_dataRow.split('\t')
    #storig the movie Id and movie names into tuples of data
    movie_Id = row[0]
    movie_Name = row[2]

    valueableData = (movie_Id, movie_Name)
    return valueableData

In [0]:
# Keeping track of movie ids along with the movie names in a list
Meta_Data_File = sc.textFile("dbfs:/FileStore/movie_metadata.tsv")
mov_Meta_data = Meta_Data_File.map(lambda x: extract_Movie_Data(x))
mov_Meta_data.collect()


Out[15]: [('975900', 'Ghosts of Mars'),
 ('3196793', 'Getting Away with Murder: The JonBenét Ramsey Mystery'),
 ('28463795', 'Brun bitter'),
 ('9363483', 'White Of The Eye'),
 ('261236', 'A Woman in Flames'),
 ('13696889', 'The Gangsters'),
 ('18998739', "The Sorcerer's Apprentice"),
 ('10408933', "Alexander's Ragtime Band"),
 ('9997961', 'Contigo y aquí'),
 ('2345652', 'City of the Dead'),
 ('175026', 'Sarah and Son'),
 ('24229100', 'Lady Snowblood 2: Love Song of Vengeance'),
 ('6631279', 'Little city'),
 ('171005', 'Henry V'),
 ('18296435', 'Aaah Belinda'),
 ('11250635', 'The Mechanical Monsters'),
 ('30388930', '1919'),
 ('77856', 'Mary Poppins'),
 ('32456683', 'Die Fahne von Kriwoj Rog'),
 ('33420460', 'Keep the Change'),
 ('175024', "The Devil's Holiday"),
 ('612710', 'New Rose Hotel'),
 ('21926710', 'White on Rice'),
 ('33427105', 'Freddy and the Song of the South Pacific'),
 ('31983669', 'Road to Life'),
 ('17715326', 'Camera Thrills'),
 ('22087420', 'Ferdinando I, re di Napoli

In [0]:
# Joining 2 rdds tfIdList and moviesMetadata to get (term,tf,idf,tf-idf,name)
tf_Idf_MetaRdd = tf_Idf_List.join(mov_Meta_data)
tf_Idf_MetaRdd.collect()

Out[16]: [('15340311',
  (('nation', 1, 2.2684671180602325, 2.2684671180602325),
   'And the Violins stopped playing')),
 ('15340311',
  (('sister', 2, 1.0709447478560365, 2.141889495712073),
   'And the Violins stopped playing')),
 ('15340311',
  (('however', 3, 0.6072856746136134, 1.82185702384084),
   'And the Violins stopped playing')),
 ('15340311',
  (('killed', 2, 0.801885637053477, 1.603771274106954),
   'And the Violins stopped playing')),
 ('15340311',
  (('go', 1, 0.6714362340022654, 0.6714362340022654),
   'And the Violins stopped playing')),
 ('15340311',
  (('two', 1, 0.5078065998369241, 0.5078065998369241),
   'And the Violins stopped playing')),
 ('15340311',
  (('daughter', 1, 0.828719030145787, 0.828719030145787),
   'And the Violins stopped playing')),
 ('15340311',
  (('family', 1, 0.7072192360181855, 0.7072192360181855),
   'And the Violins stopped playing')),
 ('15340311',
  (('approaches', 1, 1.596207179703935, 1.596207179703935),
   'And the Violins stopped play

In [0]:
# single query Keyword search:
def keywords_Search(word):
    # Filter the RDD (tf_Idf_MetaRdd) to find first element of the term's metadata (x[1][0][0]) that matches the given word.
    #sort according to the tf_idf score
    single_word_Search = tf_Idf_MetaRdd.filter(lambda x : x[1][0][0] == word).sortBy(keyfunc = lambda x: x[1][0][3], ascending = False)
    #take top 10 results
    return single_word_Search.map(lambda x :(x[1][1])).take(10)
    

In [0]:
# Mapping the RDD to extract specific elements
# and the TF-IDF score (x[1][0][2]).
idf_Multi_word_Search = tf_Idf_MetaRdd.map(lambda x: (x[1][0][0], (x[1][1], x[1][0][2])))

# Function for the weighted TF-IDF value for multiple terms
def multiple_Tf_Idf(x):
    value = 0  
    # Check if the TF-IDF score is None
    if x[1][1] is None:
        value = 0  # If TF-IDF score is None, set value to 0
    else:
        # If TF-IDF score exists, calculate the weighted value as product of term's weight and TF-IDF score 
        value = x[1][0] * x[1][1][1]
    
    # Return the keyword and the calculated value
    return (x[0], value)

# Function to calculate scores using some formula that includes term frequencies and weights
def Score(x):
    # product of the term frequency (x[1]) and the weight (x[2])
    first = x[1] * x[2]
    # square of the weight (x[2] * x[2])
    second = x[2] * x[2]
    # square of the term frequency (x[1] * x[1])
    third = x[1] * x[1]
    # Return the keyword and tuple containing the calculated values (first, second, third)
    return (x[0], (first, second, third))


In [0]:
# Function for performing a multiple keyword search based on TF-IDF scoring
def multiple_Keywords_Search(searchingTerms):
    searching_Words = searchingTerms.split(" ")  # Split the input string of search terms into individual words

    # count the frequency of each word in the search terms.
    tf_MultiSearch = sc.parallelize(searching_Words).map(lambda x: (x, 1)).reduceByKey(lambda x, y: x + y)

    # left outer join with the precomputed TF-IDF data (idf_Multi_word_Search).
    tf_Idf_MultiSearch = tf_MultiSearch.leftOuterJoin(idf_Multi_word_Search)

    # Apply 'multiple_Tf_Idf' function to each entry in the joined data to calculate the weighted TF-IDF score.
    tf_Idf_MultiSearch = tf_Idf_MultiSearch.map(lambda x: (multiple_Tf_Idf(x)))

    # Extract the term's document data and the TF-IDF score then join it with the TF-IDF scores.
    weights = tf_Idf_MetaRdd.map(lambda x: (x[1][0][0], (x[1][1], x[1][0][3])))
    weights = weights.join(tf_Idf_MultiSearch)

    # Map the joined data to create tuples of (keyword, document data, weighted TF-IDF score)
    weights = weights.map(lambda x: (x[1][0][0], x[1][0][1], x[1][1])).map(lambda x: Score(x)).reduceByKey(lambda x, y: ((x[0] + y[0], x[1] + y[1], x[2] + y[2])))

    # normalize the scores by dividing by the square root of the product of summed squares.
    # Sort by the score in descending order, and return the top 10 movie IDs.
    movies = weights.map(lambda x: (x[0], x[1][0] / (math.sqrt(x[1][1] * x[1][2])))).sortBy(keyfunc=lambda x: x[1], ascending=False).map(lambda x:x[0]).take(10)
    return movies

In [0]:
# Extracting search terms from the user's input file
userInput_SearchTerms = sc.textFile("dbfs:/FileStore/testcases-3.txt")
userInput_SearchTerms.collect()

Out[20]: ['detective',
 'thriller',
 'friendship',
 'Comedy',
 'Suspense',
 'adventure',
 'suspense crime',
 'Romantic horror',
 'Action thriller',
 'space exploration',
 'romantic comedy']

In [0]:
# Processing the user input file
updated_Input_SearchTerms = [x.lower() for x in userInput_SearchTerms.collect()]

In [0]:
# Loop through each search term in the updated list 
for searchTerm in updated_Input_SearchTerms:
    query = searchTerm.split(' ')
    # Check if the query contains only one word
    if len(query) == 1:
        print('Single query - ',searchTerm)
        # function for a single keyword search
        result = keywords_Search(searchTerm)
    else:
        print('Top 10 Documents for every single multiple search terms: ', searchTerm)
        #function for a multi-keyword search
        result = multiple_Keywords_Search(searchTerm)
    for movie in result:
        print(movie)
    print("\n")

Single query -  detective
Number Seventeen
Detective K: Secret of the Virtuous Widow
Case Closed: Jolly Rogers in the Deep Azure
What's Your Raashee?
Detectives on the Edge of a Nervous Breakdown
Baadshah
The Traveler
Case Closed: The Fourteenth Target
Blind
Let Me In


Single query -  thriller
Kyun Tum Say Itna Pyar Hai
Godfather
Man In The Mirror: The Michael Jackson Story
Left For Dead
The Case of the Frightened Lady
Adaptation
Trois 2: Pandora's Box
The Second Woman
Socket
Banking Hours 10 to 4


Single query -  friendship
Billu
Mary and Max
Such a Long Journey
Yaar Annmulle Movie
Fanfan
Nirom
Man Behind the Sun
What's Up with Love?
Heartbeats
D'Lucky Ones


Single query -  comedy
Talkin' Dirty After Dark
General Motors 50th Anniversary Show
Hollywood Outlaw Movie
Man on the Moon
Where the Truth Lies
Thousands Cheer
Mahaul Theek Hai
When Stand Up Stood Out
Funny Bones
Cinta Kura Kura


Single query -  suspense
Banking Hours 10 to 4
The Red Robin
Cairo Exit
Terror in the Aisles
Barb